In [1]:
# Importando as bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.fftpack import fft

In [2]:
# Função de criação do espectro de FFT
def espectro_fft(df, Fs):
  DF = pd.DataFrame()
  for axis in df.columns:
    x = df[axis].values
    n = np.size(x)
    fr = (Fs/2)*np.linspace(0,1,int(n/2))
    # Compute FFT
    X = fft(x)
    X_m = (2/n)*abs(X[0:np.size(fr)])
    DF[f"A_{axis}"] = X_m
  DF['Freq'] = fr
  DF.set_index('Freq', inplace=True)
  return DF

In [3]:
# Arquivos do github URLs: https://github.com/gdnpleite/ifpe_vibmec/tree/main/data
url1 = '1_DOF_01-01-2000_00-03-59.csv'
url2 = '1_DOF_01-01-2000_00-04-58.csv'
url3 = 'm1p1_04-07-2022_16-03-34.csv'
url4 = 'm1p1_04-07-2022_16-05-03.csv'
url5 = 'm1p2_04-07-2022_16-13-35.csv'
url6 = 'm1p2_04-07-2022_16-14-16.csv'
url7 = 'm2p1_04-07-2022_16-19-48.csv'
url8 = 'm2p1_04-07-2022_16-20-19.csv'
url9 = 'm3p1_04-07-2022_16-17-38.csv'
url10 = 'm3p1_04-07-2022_16-18-18.csv'
url11 = 'm3p2_04-07-2022_16-15-43.csv'
url12 = 'm4p1_04-07-2022_16-28-41.csv'
url13 = 'm4p1_04-07-2022_16-29-14.csv'
url14 = 'm4p2_04-07-2022_16-30-56.csv'
url15 = 'm4p2_04-07-2022_16-31-33.csv'
url16 = 'm4p3_04-07-2022_16-32-30.csv'
url17 = 'm4p3_04-07-2022_16-33-59.csv'
url18 = 'm4p3_04-07-2022_16-35-15.csv'
url19 = 'pend_04-07-2022_16-07-46.csv'
url20 = 'pend_04-07-2022_16-09-56.csv'
url21 = 'pend_04-07-2022_16-11-20.csv'
url22 = 'pend_04-07-2022_16-22-22.csv'
url23 = 'pend_04-07-2022_16-23-13.csv'
url24 = 'r1_04-07-2022_16-46-25.csv'
url25 = 'r1_04-07-2022_16-47-32.csv'
url26 = 'r1_04-07-2022_16-48-03.csv'
url27 = 'r130_04-07-2022_16-49-05.csv'

In [4]:
url_0 = 'https://raw.githubusercontent.com/gdnpleite/ifpe_vibmec/main/data/'+url3
url_0

'https://raw.githubusercontent.com/gdnpleite/ifpe_vibmec/main/data/m1p1_04-07-2022_16-03-34.csv'

In [5]:
df = pd.read_csv(url_0) # leitura do dado do github

In [6]:
# Criação da coluna seconds
df['seconds'] = df['micros']/1e6
df = df.set_index('seconds')
df.drop('micros', axis = 1, inplace = True)
# Conversão para m/s^2
df = df[['ax', 'ay', 'az']] * (9.81 / 16384.0)

In [7]:
# Removendo a component DC (centralizando a medição em zero)
df.ax = df.ax.values - np.mean(df.ax.values)
df.ay = df.ay.values - np.mean(df.ay.values)
df.az = df.az.values - np.mean(df.az.values)

In [8]:
df.head()

,ax,ay,az
seconds,,,
0.000000,0.021099,-0.001591,-0.006766
0.001000,0.042655,-0.025541,0.029160
0.002000,0.078580,-0.027936,0.048320
0.003000,-0.002851,-0.013566,0.041135
0.004001,-0.012431,0.053495,-0.013951


In [9]:
# calculando o dt
tam = df.shape[0]
dt_vec = (df.index[1:tam]-df.index[0:(tam-1)])
df_dt = pd.DataFrame(dt_vec)
#fig = px.line(df_dt)
#fig.show()
# cálculo da frequência de amostragem 
fs = 1/np.mean(dt_vec)
print(f"Frequência de amostragem = {fs} Hz")

Frequência de amostragem = 1000.0 Hz


In [10]:
fig = px.line(df, y = ['ax', 'ay', 'az'])
fig.update_layout(
    xaxis_title=r'Tempo [s]',
    yaxis_title=r'$\text{Amplitude } [m/s^2]$'
)
fig.show()

In [11]:
# Espectro de Fourrier
df_freq  = espectro_fft(df, fs)
#df_freq = pd.DataFrame(np.transpose([f, A]), columns=['Freq', 'Amp'])

In [12]:
df_freq.head()

,A_ax,A_ay,A_az
Freq,,,
0.00000,1.619860e-15,4.840572e-18,5.586642e-17
0.10002,2.654837e-02,2.507559e-02,1.350294e-02
0.20004,2.666943e-02,2.915325e-02,1.609244e-02
0.30006,2.618898e-02,1.060189e-02,1.420252e-02
0.40008,2.071819e-02,1.553148e-02,1.341385e-02


In [13]:
fig = px.line(df_freq, y = ['A_ax', 'A_ay', 'A_az'])
fig.update_layout(
    xaxis_title=r'Frequency [Hz]',
    yaxis_title=r'$Amplitude [m/s^2]$'
)
fig.show()